<a href="https://colab.research.google.com/github/prof-cs-ohamdas/ML_DL_IDS_UAV/blob/main/ML_IDS_UAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [2]:
# Try import pennylane (QML). If not available, we'll fall back to classical surrogate.
try:
    import pennylane as qml
    from pennylane import numpy as pnp
    from pennylane.optimize import AdamOptimizer
    QML_AVAILABLE = True
    print("[INFO] pennylane imported: QML enabled.")
except Exception:
    qml = None
    pnp = None
    AdamOptimizer = None
    QML_AVAILABLE = False
    warnings.warn(
        "pennylane not available — QML will fall back to a classical MLP surrogate. "
        "To enable QML, install pennylane: pip install pennylane"
    )

/tmp/ipython-input-3721627334.py:13: UserWarning: pennylane not available — QML will fall back to a classical MLP surrogate. To enable QML, install pennylane: pip install pennylane
  warnings.warn(


Data Set load from Github

In [3]:
import pandas as pd

url = "https://raw.githubusercontent.com/prof-cs-ohamdas/ML_DL_IDS_UAV/main/drone_communication_dataset.csv"
df = pd.read_csv(url)

print(f"[INFO] Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")


[INFO] Dataset loaded: 45289 rows, 35 columns


In [4]:
!git clone https://github.com/prof-cs-ohamdas/ML_DL_IDS_UAV.git
%cd ML_DL_IDS_UAV

import pandas as pd
df = pd.read_csv("drone_communication_dataset.csv")
print(f"[INFO] Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")


fatal: destination path 'ML_DL_IDS_UAV' already exists and is not an empty directory.
/content/ML_DL_IDS_UAV
[INFO] Dataset loaded: 45289 rows, 35 columns


In [5]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

df


,timestamp,signal_strength,packet_loss_rate,round_trip_time,communication_protocol,frequency_band,encryption_type,drone_gps_coordinates,altitude,speed_trajectory,...,intrusion_detection_flags,temporal_patterns,label_normal,label_spoofing,label_mitm,label_ddos,label_gps_spoofing,label_malware,label_jamming,label_protocol_exploit
0,2019-11-01 00:00:00,-60,0,50,ZigBee,5.0,AES,"(-11.69864152850468, 86.89778588010734)",291,60,...,0,0,0,0,0,0,0,0,0,0
1,2019-11-01 01:00:00,-64,0,50,LoRa,2.4,AES,"(66.98918792506822, -140.91891468822496)",100,48,...,0,0,0,0,0,0,0,0,0,0
2,2019-11-01 02:00:00,-60,0,194,ZigBee,2.4,AES,"(-31.711586784829194, 36.5632095665857)",226,30,...,0,0,0,0,0,0,0,0,0,0
3,2019-11-01 03:00:00,-60,0,50,Wi-Fi,2.4,AES,"(3.5943733073538198, 74.7992608739666)",100,30,...,0,0,1,0,0,0,0,0,1,0
4,2019-11-01 04:00:00,-69,0,50,Wi-Fi,2.4,Plain-text,"(41.4738797417983, -100.00817301602393)",100,30,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45284,2024-12-30 20:00:00,-60,0,50,LoRa,915.0,Plain-text,"(62.250807704044774, -151.05594484833514)",100,30,...,0,0,0,0,0,0,0,0,0,0
45285,2024-12-30 21:00:00,-66,0,50,ZigBee,2.4,RSA,"(-44.694536026285874, 164.52171076223817)",100,30,...,0,0,0,0,0,0,0,0,0,0
45286,2024-12-30 22:00:00,-60,12,50,ZigBee,2.4,RSA,"(-89.91726166541113, -120.42483221247912)",100,30,...,0,1,0,0,0,1,1,0,1,0
45287,2024-12-30 23:00:00,-60,0,50,Wi-Fi,2.4,Plain-text,"(-55.27640423530802, 61.96296050101765)",100,30,...,0,1,0,0,0,0,0,0,0,0


In [6]:
# -------------------------
# 2. Inspect label columns
# -------------------------
label_cols = [c for c in df.columns if c.startswith("label_")]
if not label_cols:
    raise ValueError("No label_* columns found in dataset. Need label columns for supervised training.")
print(f"[INFO] label columns detected: {label_cols}")


[INFO] label columns detected: ['label_normal', 'label_spoofing', 'label_mitm', 'label_ddos', 'label_gps_spoofing', 'label_malware', 'label_jamming', 'label_protocol_exploit']


In [7]:
# -------------------------
# 3. Create multi-class target (attack_type)
# -------------------------
# Each row is assumed to have a one-hot label across label_* columns; idxmax gives the label name.
df["attack_type_raw"] = df[label_cols].idxmax(axis=1)  # e.g. 'label_ddos'
df["attack_type"] = df["attack_type_raw"].str.replace("label_", "", regex=False)


In [8]:
# -------------------------
# 4. Create binary target for QML: normal vs attack
#    Strategy: try to find explicit 'normal' label; otherwise assume the most frequent label is 'normal'
# -------------------------
normal_label_col = next((c for c in label_cols if "normal" in c.lower()), None)
if normal_label_col:
    print(f"[INFO] Found explicit normal label column: {normal_label_col}")
    df["is_attack"] = (df[normal_label_col] == 0).astype(int)  # 1 -> attack, 0 -> normal
else:
    # No explicit 'label_normal' column. We'll assume the most frequent label is 'normal'.
    most_freq_label = df["attack_type"].mode().iloc[0]
    print(f"[WARN] No explicit 'label_normal' column. Treating most frequent label '{most_freq_label}' as NORMAL.")
    df["is_attack"] = (df["attack_type"] != most_freq_label).astype(int)


[INFO] Found explicit normal label column: label_normal


In [9]:
# -------------------------
# 5. Drop non-useful columns
# -------------------------
cols_to_drop = []
for c in ["timestamp", "drone_gps_coordinates", "attack_type_raw"]:
    if c in df.columns:
        cols_to_drop.append(c)

# We'll keep numeric/categorical features; drop label_* originals after deriving targets
cols_to_drop += label_cols
df = df.drop(columns=cols_to_drop)
print(f"[INFO] Dropped columns: {cols_to_drop}")


[INFO] Dropped columns: ['timestamp', 'drone_gps_coordinates', 'attack_type_raw', 'label_normal', 'label_spoofing', 'label_mitm', 'label_ddos', 'label_gps_spoofing', 'label_malware', 'label_jamming', 'label_protocol_exploit']


In [10]:
# -------------------------
# 6. Prepare features and encode categorical columns
# -------------------------
# Identify object (text) columns except 'attack_type' (we'll encode it separately)
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
if "attack_type" in categorical_cols:
    categorical_cols.remove("attack_type")

print(f"[INFO] Categorical columns to encode: {categorical_cols}")

# One-hot encode categorical cols (safe for models)
df_enc = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Final feature matrix X and labels y_multi (multi-class) and y_bin (binary)
X = df_enc.drop(columns=["attack_type", "is_attack"])
y_multi = df_enc["attack_type"]
y_bin = df_enc["is_attack"]

# Encode multi-class labels to integers
le = LabelEncoder()
y_multi_enc = le.fit_transform(y_multi)
print(f"[INFO] Multi-class labels found: {list(le.classes_)}")

[INFO] Categorical columns to encode: ['communication_protocol', 'encryption_type']
[INFO] Multi-class labels found: ['ddos', 'gps_spoofing', 'jamming', 'malware', 'mitm', 'normal', 'protocol_exploit', 'spoofing']


In [11]:
# -------------------------
# 7. Train/test split
# -------------------------
# Classical (multi-class) models
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X, y_multi_enc, test_size=0.20, random_state=42, stratify=y_multi_enc
)

# QML (binary) - use the same feature set but we'll restrict to smaller feature subset for QNN
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(
    X, y_bin, test_size=0.20, random_state=42, stratify=y_bin
)

# Standardize (important for SVM and many models)
scaler = StandardScaler()
X_train_m_scaled = scaler.fit_transform(X_train_m)
X_test_m_scaled = scaler.transform(X_test_m)

X_train_b_scaled = scaler.transform(X_train_b)  # use same scaler (fit on multi-class training)
X_test_b_scaled = scaler.transform(X_test_b)

print(f"[INFO] Feature dimension: {X_train_m_scaled.shape[1]}")


[INFO] Feature dimension: 27


In [12]:
# -------------------------
# 8. Train classical multi-class models
# -------------------------
models = {
    "Logistic Regression": LogisticRegression(max_iter=2000, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM": SVC(kernel="rbf", probability=True, random_state=42)
}

results = {}
for name, model in models.items():
    print(f"\n[TRAIN] {name} (multi-class)...")
    model.fit(X_train_m_scaled, y_train_m)
    preds = model.predict(X_test_m_scaled)
    acc = accuracy_score(y_test_m, preds)
    results[name] = {"model": model, "accuracy": acc, "y_pred": preds}
    print(f"[RESULT] {name} accuracy: {acc:.4f}")
    print(classification_report(y_test_m, preds, target_names=le.classes_))



[TRAIN] Logistic Regression (multi-class)...
[RESULT] Logistic Regression accuracy: 0.7250
                  precision    recall  f1-score   support

            ddos       0.00      0.00      0.00       605
    gps_spoofing       0.00      0.00      0.00       286
         jamming       0.00      0.00      0.00       429
         malware       0.00      0.00      0.00       409
            mitm       0.00      0.00      0.00       237
          normal       0.72      1.00      0.84      6567
protocol_exploit       0.00      0.00      0.00       118
        spoofing       0.00      0.00      0.00       407

        accuracy                           0.72      9058
       macro avg       0.09      0.12      0.11      9058
    weighted avg       0.53      0.72      0.61      9058


[TRAIN] KNN (multi-class)...


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[RESULT] KNN accuracy: 0.6862
                  precision    recall  f1-score   support

            ddos       0.09      0.03      0.05       605
    gps_spoofing       0.03      0.01      0.01       286
         jamming       0.05      0.01      0.02       429
         malware       0.07      0.01      0.02       409
            mitm       0.04      0.00      0.01       237
          normal       0.73      0.94      0.82      6567
protocol_exploit       0.00      0.00      0.00       118
        spoofing       0.04      0.00      0.00       407

        accuracy                           0.69      9058
       macro avg       0.13      0.13      0.12      9058
    weighted avg       0.54      0.69      0.60      9058


[TRAIN] SVM (multi-class)...
[RESULT] SVM accuracy: 0.7250
                  precision    recall  f1-score   support

            ddos       0.00      0.00      0.00       605
    gps_spoofing       0.00      0.00      0.00       286
         jamming       0.00      0.0

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
# -------------------------
# 9. QML binary classifier (QNN) OR MLP surrogate if pennylane unavailable
#    We'll use a small subset of features for the QNN (e.g., first 3 principal features)
# -------------------------
# Choose k features for the QNN (low-dim). We'll pick the first 3 numeric columns (after encoding).
k_qnn = 3
if X_train_b_scaled.shape[1] < k_qnn:
    k_qnn = X_train_b_scaled.shape[1]

X_train_q = X_train_b_scaled[:, :k_qnn]
X_test_q = X_test_b_scaled[:, :k_qnn]

if QML_AVAILABLE:
    print("\n[QML] Training Quantum Neural Network (binary)...")
    n_qubits = k_qnn
    dev = qml.device("default.qubit", wires=n_qubits)

    # Variational circuit: AngleEmbedding + StronglyEntanglingLayers
    def quantum_circuit(params, x=None):
        # angle embedding expects length==n_qubits
        x_padded = np.resize(x, n_qubits)  # truncate or pad
        qml.AngleEmbedding(x_padded, wires=range(n_qubits))
        qml.StronglyEntanglingLayers(params, wires=range(n_qubits))
        return qml.expval(qml.PauliZ(0))

    n_layers = 2
    params_shape = qml.StronglyEntanglingLayers.shape(n_layers=n_layers, n_wires=n_qubits)
    params = pnp.random.random(params_shape, requires_grad=True)

    @qml.qnode(dev, interface="autograd")
    def qnode(params, x):
        return quantum_circuit(params, x)

    def qnn_predict_prob(params, x):
        # map expectation [-1,1] -> [0,1]
        return (qnode(params, x) + 1) / 2

    # Prepare pnp arrays
    X_train_q_p = pnp.array(X_train_q, requires_grad=False)
    X_test_q_p = pnp.array(X_test_q, requires_grad=False)
    y_train_q_p = pnp.array(y_train_b.values, requires_grad=False)

    opt = AdamOptimizer(stepsize=0.2)

    def cost(params, X, y):
        preds = pnp.array([qnn_predict_prob(params, X[i]) for i in range(len(X))])
        return pnp.mean((preds - y) ** 2)

    epochs = 15
    for epoch in range(epochs):
        params = opt.step(lambda v: cost(v, X_train_q_p, y_train_q_p), params)
        if (epoch + 1) % 5 == 0 or epoch == 0:
            c = cost(params, X_train_q_p, y_train_q_p)
            print(f" Epoch {epoch+1}/{epochs}  cost: {c:.4f}")

    # Evaluate QNN
    q_probs = [qnn_predict_prob(params, x) for x in X_test_q_p]
    q_preds = [1 if float(p) > 0.5 else 0 for p in q_probs]
    q_acc = accuracy_score(y_test_b, q_preds)
    print(f"\n[QNN RESULT] Binary QNN accuracy: {q_acc:.4f}")
    print(classification_report(y_test_b, q_preds))
    results["Quantum QNN (binary)"] = {"model": "pennylane_qnn", "accuracy": q_acc, "y_pred": q_preds}
else:
    # Fallback: small MLP surrogate
    print("\n[QML fallback] pennylane unavailable: training MLP surrogate (binary)...")
    mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)
    mlp.fit(X_train_q, y_train_b)
    mlp_preds = mlp.predict(X_test_q)
    mlp_acc = accuracy_score(y_test_b, mlp_preds)
    print(f"[MLP surrogate] Binary accuracy: {mlp_acc:.4f}")
    print(classification_report(y_test_b, mlp_preds))
    results["QNN surrogate (MLP)"] = {"model": mlp, "accuracy": mlp_acc, "y_pred": mlp_preds}



[QML fallback] pennylane unavailable: training MLP surrogate (binary)...
[MLP surrogate] Binary accuracy: 0.8982
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       922
           1       0.90      1.00      0.95      8136

    accuracy                           0.90      9058
   macro avg       0.45      0.50      0.47      9058
weighted avg       0.81      0.90      0.85      9058



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
# -------------------------
# 10. Sanity checks
# -------------------------
print("\n[Sanity Checks]")
for name, info in results.items():
    pred = info["y_pred"]
    if len(pred) != X_test_m_scaled.shape[0] and ("binary" not in name.lower() and "qnn" not in name.lower() and "surrogate" not in name.lower()):
        print(f" [WARN] {name} prediction length ({len(pred)}) != multi-class test size ({X_test_m_scaled.shape[0]})")
    else:
        print(f" [OK] {name} predictions ready (accuracy {info['accuracy']:.4f})")

print("\n[Done] IDS training & evaluation complete.")



[Sanity Checks]
 [OK] Logistic Regression predictions ready (accuracy 0.7250)
 [OK] KNN predictions ready (accuracy 0.6862)
 [OK] SVM predictions ready (accuracy 0.7250)
 [OK] QNN surrogate (MLP) predictions ready (accuracy 0.8982)

[Done] IDS training & evaluation complete.


**Graph Neural Networks (GNNs)**

In [15]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

In [16]:
# The data loading and preprocessing have been done in cell hAaESYBRdi5-,
# resulting in X and y variables. We will use those here.
# No preprocessing needed in this cell.

In [17]:
# -------------------------
# 2. Build Graph
# -------------------------
# For simplicity: connect each node to its k nearest neighbors
from sklearn.neighbors import kneighbors_graph
import numpy as np

k = 5
adj = kneighbors_graph(X, k, mode="connectivity", include_self=False)
edge_index = torch.tensor(np.array(adj.nonzero()), dtype=torch.long)

# Convert to PyTorch Geometric Data
data = Data(
    x=torch.tensor(X.values, dtype=torch.float), # Use .values here
    y=torch.tensor(y, dtype=torch.long),
    edge_index=edge_index
)

# Train/test split
num_nodes = data.num_nodes
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_idx, test_idx = train_test_split(range(num_nodes), test_size=0.2, stratify=y, random_state=42)
train_mask[train_idx] = True
test_mask[test_idx] = True
data.train_mask = train_mask
data.test_mask = test_mask

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [ ]:
# -------------------------
# 3. Define GNN Model
# -------------------------
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

model = GNN(in_channels=data.num_node_features, hidden_channels=32, out_channels=len(le.classes_))

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = torch.nn.CrossEntropyLoss()


In [ ]:
# -------------------------
# 4. Train
# -------------------------
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [ ]:
# -------------------------
# 5. Test
# -------------------------
model.eval()
preds = out.argmax(dim=1)
correct = preds[data.test_mask].eq(data.y[data.test_mask]).sum().item()
acc = correct / data.test_mask.sum().item()
print(f"Test Accuracy: {acc:.4f}")

**XGBoost, LightGBM, CatBoost**

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Try importing XGBoost, LightGBM, CatBoost, and PennyLane
try:
    from xgboost import XGBClassifier
    xgb_available = True
except ImportError:
    print("Warning: XGBoost not installed. Skipping XGBoost.")
    xgb_available = False

try:
    from lightgbm import LGBMClassifier
    lgbm_available = True
except ImportError:
    print("Warning: LightGBM not installed. Skipping LightGBM.")
    lgbm_available = False

try:
    from catboost import CatBoostClassifier
    catboost_available = True
except ImportError:
    print("Warning: CatBoost not installed. Skipping CatBoost.")
    catboost_available = False

try:
    import pennylane as qml
    from pennylane import numpy as pnp
    pennylane_available = True
except ImportError:
    print("Warning: PennyLane not installed. Quantum ML will be skipped.")
    pennylane_available = False

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [19]:
# Try importing XGBoost
try:
    from xgboost import XGBClassifier
    xgb_available = True
except ImportError:
    print("Warning: XGBoost not installed. Skipping XGBoost.")
    xgb_available = False

In [20]:
# Try importing LightGBM
try:
    from lightgbm import LGBMClassifier
    lgbm_available = True
except ImportError:
    print("Warning: LightGBM not installed. Skipping LightGBM.")
    lgbm_available = False

In [21]:
# Try importing CatBoost
try:
    from catboost import CatBoostClassifier
    catboost_available = True
except ImportError:
    print("Warning: CatBoost not installed. Skipping CatBoost.")
    catboost_available = False


In [22]:
# Try importing PennyLane for QML
try:
    import pennylane as qml
    from pennylane import numpy as pnp
    pennylane_available = True
except ImportError:
    print("Warning: PennyLane not installed. Quantum ML will be skipped.")
    pennylane_available = False


In [23]:
# -------------------------
# 2. Preprocess dataset
# -------------------------
# Drop non-useful columns (timestamp, GPS coordinates, etc.)
drop_cols = ["timestamp", "drone_gps_coordinates"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])


In [24]:
import pandas as pd

# Load dataset
url = "https://raw.githubusercontent.com/prof-cs-ohamdas/ML_DL_IDS_UAV/main/drone_communication_dataset.csv"
df = pd.read_csv(url)

# Inspect columns
print("Columns:", df.columns.tolist()[:20])  # show first 20 columns

# Handle label column(s)
if "label" in df.columns:
    # Case 1: dataset has a single label column
    df["attack_type"] = df["label"]

else:
    # Case 2: dataset has multiple one-hot encoded labels
    label_cols = [c for c in df.columns if "label" in c.lower()
                  or c.lower() in ["normal", "dos", "probe", "u2r", "r2l"]]

    if len(label_cols) == 0:
        raise ValueError("No label columns found. Please check dataset columns.")

    # Merge into one target column
    df["attack_type"] = df[label_cols].idxmax(axis=1)
    df["attack_type"] = df["attack_type"].str.replace("label_", "", regex=False)
    df = df.drop(columns=label_cols)

print("\n[INFO] ✅ attack_type column created successfully")
print(df[["attack_type"]].head())



Columns: ['timestamp', 'signal_strength', 'packet_loss_rate', 'round_trip_time', 'communication_protocol', 'frequency_band', 'encryption_type', 'drone_gps_coordinates', 'altitude', 'speed_trajectory', 'transmission_power', 'message_authentication_status', 'session_key_validity', 'signal_noise_ratio', 'sequence_number_gap', 'drone_identification', 'data_rate', 'network_traffic_volume', 'gps_signal_integrity', 'uplink_downlink_quality']

[INFO] ✅ attack_type column created successfully
  attack_type
0      normal
1      normal
2      normal
3      normal
4      normal


In [25]:
# Encode categorical features
categorical_cols = df.select_dtypes(include=["object"]).columns.drop("attack_type")
df = pd.get_dummies(df, columns=categorical_cols)


In [26]:
# Encode target
le = LabelEncoder()
df["attack_type"] = le.fit_transform(df["attack_type"])

In [29]:
# -------------------------
# 3. Train/test split
# -------------------------
# X and y are already prepared in the preceding preprocessing cell.
# Use X and y_multi_enc for multi-class models.

X_train, X_test, y_train, y_test = train_test_split(
    X, y_multi_enc, test_size=0.2, random_state=42, stratify=y_multi_enc
)

# Standardize - X is already scaled in the preceding preprocessing cell.
# No need to standardize again here.
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

**Federated Learning (FL)**

In [30]:
# Federated Learning IDS Example for UAV 5G Network
# Using Flower (flwr) and Logistic Regression

import flwr as fl
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [2]:
# -------------------------
# 4. Start Federated Learning
# -------------------------
from flwr.server.server import ServerConfig
import flwr as fl # Import flwr here as well
import numpy as np # Import numpy
import os # Import os for file path handling

# Define the number of clients
num_clients = 3 # Use the existing variable

# Prepare data for clients (example: split X_train, y_train)
# In a real FL scenario, each client would have its own data
# For simulation, we'll split the training data further.
# The train/test split was already done in previous cells, resulting in X_train_m, y_train_m, X_test_m, y_test_m
# and X_train_b, y_train_b, X_test_b, y_test_b.

# We'll use the binary split data for FL for simplicity (is_attack target)
# Let's split X_train_b and y_train_b into num_clients partitions

X_train_b_split = np.array_split(X_train_b, num_clients)
y_train_b_split = np.array_split(y_train_b, num_clients)

# Save data splits to temporary files
temp_dir = "/tmp/fl_data"
os.makedirs(temp_dir, exist_ok=True)
client_data_paths = {}
for i in range(num_clients):
    x_path = os.path.join(temp_dir, f"client_{i}_X_train.npy")
    y_path = os.path.join(temp_dir, f"client_{i}_y_train.npy")
    np.save(x_path, X_train_b_split[i])
    np.save(y_path, y_train_b_split[i])
    client_data_paths[str(i)] = (x_path, y_path)

print(f"[INFO] Data splits saved to {temp_dir} for {num_clients} clients.")


def client_fn(cid: str):
    """Returns a Flower client instance."""
    # Data is now loaded within the client instance
    return UAVClient(cid, client_data_paths[cid]) # Pass file paths


strategy = fl.server.strategy.FedAvg()
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=num_clients,
    config=ServerConfig(num_rounds=5),
    strategy=strategy,
)

NameError: name 'X_train_b' is not defined

In [3]:
# -------------------------
# 6. Prepare features and encode categorical columns
# -------------------------
# Identify object (text) columns except 'attack_type' (we'll encode it separately)
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
if "attack_type" in categorical_cols:
    categorical_cols.remove("attack_type")

print(f"[INFO] Categorical columns to encode: {categorical_cols}")

# One-hot encode categorical cols (safe for models)
df_enc = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Final feature matrix X and labels y_multi (multi-class) and y_bin (binary)
X = df_enc.drop(columns=["attack_type", "is_attack"])
y_multi = df_enc["attack_type"]
y_bin = df_enc["is_attack"]

# Encode multi-class labels to integers
le = LabelEncoder()
y_multi_enc = le.fit_transform(y_multi)
print(f"[INFO] Multi-class labels found: {list(le.classes_)}")

NameError: name 'df' is not defined

In [33]:
from collections import OrderedDict
from typing import Dict, List, Tuple

from flwr.common import NDArrays, Scalar
from flwr.client import NumPyClient
import numpy as np # Ensure numpy is imported
import os # Import os for path joining

# Define the Flower client
class UAVClient(NumPyClient):
    def __init__(self, cid: str, data_paths: Tuple[str, str]):
        self.cid = cid
        self.x_path, self.y_path = data_paths
        # Load data from files
        try:
            self.X_train = np.load(self.x_path)
            self.y_train = np.load(self.y_path)
            print(f"[INFO] Client {self.cid}: Data loaded successfully from files.")
        except Exception as e:
            print(f"[ERROR] Client {self.cid}: Failed to load data from files: {e}")
            self.X_train = np.array([]) # Initialize with empty arrays on error
            self.y_train = np.array([])

        self.model = LogisticRegression(max_iter=1000, solver="liblinear") # Example model

        # Partition data for this client
        # This is a simplification; in real FL, data would be inherently partitioned
        # For simulation, we split the overall data. This was done in previous steps.

    def get_parameters(self, config: Dict[str, Scalar]) -> NDArrays:
        """Get parameters of the local model."""
        if self.model.coef_ is None or self.model.intercept_ is None:
            # Initialize model if not already fitted (required for .coef_ and .intercept_)
            # A dummy fit is needed to initialize model parameters
            if self.X_train.size > 0: # Only fit if data is loaded
                n_features = self.X_train.shape[1]
                # Create dummy data with the correct number of features
                dummy_X = np.zeros((1, n_features))
                dummy_y = np.zeros(1) # Assuming binary classification for simplicity
                try:
                    self.model.fit(dummy_X, dummy_y)
                except Exception as e:
                    print(f"Error during dummy fit for client {self.cid}: {e}")
                    return [] # Return empty if initialization fails
            else:
                 print(f"[WARN] Client {self.cid}: Skipping dummy fit, no training data loaded.")
                 return [] # Return empty if no data

        return [self.model.coef_, self.model.intercept_]


    def fit(self, parameters: NDArrays, config: Dict[str, Scalar]) -> Tuple[NDArrays, int, Dict[str, Scalar]]:
        """Train the model on the local data."""
        if self.X_train.size == 0:
             print(f"[WARN] Client {self.cid}: Skipping fit, no training data loaded.")
             return self.get_parameters({}), 0, {} # Return current params, 0 examples

        # Set model parameters
        self.model.coef_ = parameters[0]
        self.model.intercept_ = parameters[1]

        # Train the model
        self.model.fit(self.X_train, self.y_train)

        # Return updated model parameters and number of examples
        return self.get_parameters({}), len(self.X_train), {}

    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar]) -> Tuple[float, int, Dict[str, Scalar]]:
        """Evaluate the model on the local data."""
        if self.X_train.size == 0:
             print(f"[WARN] Client {self.cid}: Skipping evaluate, no training data loaded.")
             return 0.0, 0, {"accuracy": 0.0} # Return dummy results

        # Set model parameters
        self.model.coef_ = parameters[0]
        self.model.intercept_ = parameters[1]

        # Evaluate the model
        loss = 0 # In a real scenario, calculate loss
        accuracy = self.model.score(self.X_train, self.y_train) # Evaluate on training data for simplicity

        # Return loss, number of examples, and metrics
        return loss, len(self.X_train), {"accuracy": accuracy}

# The data splitting and saving is now done in the cell that starts the simulation.
# The client_data variable defined here previously is no longer needed in the main process.
# print(f"[INFO] Data split into {num_clients} partitions for FL clients.")

[INFO] Data split into 3 partitions for FL clients.


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
